In [3]:
import pandas as pd # 数据存储
import requests # 网页内容获取
import re # 解析数据
from lxml import etree # 解析数据
import random
import time # 反反爬
import os
from requests.adapters import HTTPAdapter
import urllib3
import urllib3.contrib.pyopenssl

start_time = time.perf_counter()

def ua():
    """随机获取一个浏览器用户信息"""

    user_agents = [
        'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
        'Opera/9.25 (Windows NT 5.1; U; en)',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
        'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
        'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
        'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
        'Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7',
        'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0',

    ]

    agent = random.choice(user_agents)

    return {
        'User-Agent': agent
    }


def get(url):
    """
    获取网页源码
    url: 目标网页的地址
    return:网页源码
    """
    s = requests.Session()
    s.mount('http://', HTTPAdapter(max_retries=3))  # 访问http协议时，设置重传请求最多三次
    s.mount('https://', HTTPAdapter(max_retries=3))  # 访问https协议时，设置重传请求最多三次
    s.keep_alive = False  # 关闭多余连接
    urllib3.disable_warnings()
    #requests.packages.urllib3.disable_warnings()
    res = requests.get(url=url, headers=ua(), verify=False, timeout=(5, 7))
    return res.text



def get_url(res_text):
    """
    获取源码中每个二手房详情页的url
    res_text:网页源码
    return:列表形式的30个二手房详情页的url
    """
    re_f = '<a class="noresultRecommend img LOGCLICKDATA" href="(.*?)" target="_blank"'
    url_list = re.findall(re_f, res_text)
    return url_list


def get_html(url):
    try:
        return get(url)
    except:
        secends = random.randint(30, 60)
        print('出现异常，休息一会儿......')
        print(str(secends) + '秒后继续')
        time.sleep(secends)  # 暂停10~30分钟，然后再尝试一次
        try:
            print('重试请求......')
            return get(url)
        except:
            print('出现异常，休息一会儿......')
            print('10分钟后继续')
            time.sleep(160)  # 暂停30分钟
            try:
                print('重试请求......')
                return get(url)
            except:
                print('出现异常，休息一会儿......')
                print('30分钟后继续')
                time.sleep(600)  # 暂停30分钟
                return get(url)

def get_page(res_text):

    #re_f = '<a class="img" href="(.*?)" target="_blank"'
    re_f ='"totalPage":(.*?),"curPage"'
    page_list = re.findall(re_f, res_text)
    if len(page_list) == 0:
        return 0
    else:
        page = int(page_list[0])
        return page

def get_dijishi_url(sheng,small_city,start_pg=1):
    #url='https://bj.lianjia.com/xiaoqu/'+dongcheng+''/'
    url="https://"+sheng+".lianjia.com/ershoufang/"+small_city+'/'
    resp=get(url)
    end_pg=get_page(resp)
    all_url=[]
    for i in range(start_pg,end_pg+1):
        new_url = url + 'pg' + str(i) + '/'
        new_resp=get(new_url)
        url_list=get_url(new_resp)
        cleaned_url = list(filter(lambda x: 'https://' in x, url_list))
        print(len(cleaned_url))
        all_url.extend(cleaned_url)
        time.sleep(random.random())
    print(small_city+'有',len(all_url),'个')
    return all_url

def get_city(sheng):
    url='https://'+sheng+'.lianjia.com/ershoufang/'
    resp=get(url)
    re_f = ' <a href="/ershoufang/(.*?)/"  title="'
    city_list = re.findall(re_f, resp)
    return city_list

# def get_city_small(city):
#     url='https://'+sheng+'.lianjia.com/xiaoqu/'+city+'/'
#     resp = get(url)
#     re_f = '<a href="/xiaoqu/(.*?)/" >'
#     city_small_list = re.findall(re_f, resp)
#     print(city_small_list)
#     return city_small_list

def get_data(url):
    """获取每一页房屋的详细数据"""
    res_text = get_html(url)
    #获得经纬度
    re_f = "resblockPosition:'(.*?)',"
    lon_lat = re.findall(re_f, res_text)

    res_text = etree.HTML(res_text)
    data = []

    # 获取小区名字
    name = res_text.xpath("/html/body/div[5]/div[2]/div[4]/div[1]/a[1]/text()")
    # 获取小区位置
    address = res_text.xpath("/html/body/div[5]/div[2]/div[4]/div[2]/span[2]/a[1]/text()")
    # 获取房屋的单价
    price = res_text.xpath("/html/body/div[5]/div[2]/div[2]/div/div[1]/div[1]/span/text()")
    # 房屋总价
    tatal_price = res_text.xpath("/html/body/div[5]/div[2]/div[2]/div/span[1]/text()")
    # 房屋面积
    house_size = res_text.xpath("/html/body/div[5]/div[2]/div[3]/div[3]/div[1]/text()")
    
    try:
        name = name[0]
        address =  address[0]
        price = price[0]
        tatal_price = tatal_price[0]
        house_size = house_size[0]

        lon_lat = lon_lat[0]
    except:
        name = "无"
        address = "无"
        price = "无"
        tatal_price = "无"
        house_size = "无"

        lon_lat = "无"

        
    # 讲每一条数据加入data中
    data.append(dict(zip(['url', 'name', 'address', 'price', 'tatal_price', 'house_size',
                          'lon_lat'],
                         [url, name, address, price, tatal_price, house_size,
                           lon_lat])))
    #print(data)
    return data


def main(sheng):
    # 文件保存路径
    filename = 'D:\\lianjiapachong\\fangyuan\\' + sheng + '_ershoufang.csv'
    print('房源-' + sheng + '下载')
    city_list = get_city(sheng)
    print(city_list)
    for i in range(0, len(city_list)):
    #for i in range(0, 3):
        data = []
        city_url = get_dijishi_url(sheng,city_list[i])
        for j in range(0, len(city_url)):
            data.extend(get_data(city_url[j]))
            # time.sleep(random.random())
            time.sleep(random.random())
            print(j)
        # pd.DataFrame(data).to_csv(filename, encoding="utf_8_sig")
        pd.DataFrame(data).to_csv('D:\\lianjiapachong\\fangyuan\\'+ sheng + '\\' + city_list[i] + '_ershoufang.csv', encoding="utf_8_sig")
        print(city_list[i] + '数据已保存')
        time.sleep(random.randint(10,30))

        # 
sheng=['xsbn' ]
for i in range(0,len(sheng)):
    main(sheng[i])
    print(sheng[i],"已保存")


end_time = time.perf_counter()
print('The total run time: ', end_time-start_time)

房源-xsbn下载
['jinghongshi', 'menghaixian', 'menglaxian']
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
28
jinghongshi有 778 个
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
23